In [2]:
import numpy as np
import multiprocessing as mp
import random
import re
import sys
import scipy.sparse

In [3]:
def softprojc(vec, i, c = -1e-5):
    return(np.where(vec < c, c, vec))
def softproji(vec, i):
    return(np.where(vec < 0, (-1 / np.sqrt(i)), vec))

In [4]:
# mode 1 samples rows
def weightsample(data, mode):
    prob = np.linalg.norm(data, axis=mode)
    return(prob / sum(prob))

In [5]:
def als(data, k, niter, reinit = 1):
    # set to negative one so we can guarantee an update for the first init
    finalerror = -1
    
    # need to compare final error to overall best and store the overall best
    seqerror = np.empty(niter)
    lowesterror = np.empty(1)
    
    # store overall best factor matrices
    lbest = np.random.rand(data.shape[0], k)
    rbest = np.random.rand(k, data.shape[1])
    
    for j in np.arange(reinit):
        # randomly initialize the factor matrices
        lfactor = np.random.rand(data.shape[0], k)
        rfactor = np.random.rand(k, data.shape[1])

        for i in np.arange(niter):            
            # sample random row or column
            row = np.random.randint(data.shape[0])
            col = np.random.randint(data.shape[1])
            
            # perform linear reg update 
            rfactor[:, col] = np.matmul(np.linalg.pinv(lfactor), data[:, col])
            lfactor[row, :] = np.matmul(data[row, :], np.matmul(rfactor.T, np.linalg.inv(np.matmul(rfactor, rfactor.T))))
            # calculate error after update
            seqerror[i] = np.linalg.norm(data - np.matmul(lfactor, rfactor)) / np.linalg.norm(data)
        # update after first init
        if (finalerror == -1):
            lowesterror = seqerror
            lbest = lfactor
            rbest = rfactor
        # if not first, only update if final error is lower than overall best
        elif (finalerror > seqerror[niter - 1]):
            finalerror = seqerror[niter - 1]
            lowesterror = seqerror
            lbest = lfactor
            rbest = rfactor
    return(lbest, rbest, lowesterror)

In [6]:
def rk(data, k, niter, kacziter, reinit = 1):
    # set to negative one so we can guarantee an update for the first init
    finalerror = -1
    
    # need to compare final error to overall best and store the overall best
    seqerror = np.empty(niter)
    lowesterror = np.empty(1)
    
    # store overall best factor matrices
    lbest = np.random.rand(data.shape[0], k)
    rbest = np.random.rand(k, data.shape[1])
    
    for l in np.arange(reinit):
        # randomly initialize the factor matrices
        lfactor = np.random.rand(data.shape[0], k)
        rfactor = np.random.rand(k, data.shape[1])
        
        # outer loop for number of iterations 
        for i in np.arange(niter):
            approx = np.matmul(lfactor, rfactor)
            
            # weighted sampling of row and column from data approx matrix
            row = np.random.choice(data.shape[0], size = 1, p = weightsample(approx, 1))
            col = np.random.choice(data.shape[1], size = 1, p = weightsample(approx, 0))
            
            # inner loop for number of RK iterations
            for j in np.arange(kacziter):
                # sample index for entry of data matrix
                kaczrow = np.random.choice(lfactor.shape[0], size = 1, p = weightsample(lfactor, 1))
                kaczcol = np.random.choice(rfactor.shape[1], size = 1, p = weightsample(rfactor, 0))

                # compute RK step
                lfactor[row, :] = lfactor[row, :] + (data[row, kaczcol] - np.matmul(lfactor[row, :], rfactor[:, kaczcol])) / (np.linalg.norm(rfactor[:, kaczcol])**2) * rfactor[:, kaczcol].T 
                rfactor[:, col] = rfactor[:, col] + (data[kaczrow, col] - np.matmul(lfactor[kaczrow, :], rfactor[:, col])) / (np.linalg.norm(lfactor[kaczrow, :])**2) * lfactor[kaczrow, :].T
     
            # calculate error after update
            seqerror[i] = np.linalg.norm(data - np.matmul(lfactor, rfactor)) / np.linalg.norm(data)
        # update after first init
        if (finalerror == -1):
            lowesterror = seqerror
            lbest = lfactor
            rbest = rfactor
        # if not first, only update if final error is lower than overall best
        elif (finalerror > seqerror[niter - 1]):
            finalerror = seqerror[niter - 1]
            lowesterror = seqerror
            lbest = lfactor
            rbest = rfactor
    return(lbest, rbest, lowesterror)

In [7]:
def brk(data, k, s, niter, kacziter, reinit = 1):
    # set to negative one so we can guarantee an update for the first init
    finalerror = -1
    
    # need to compare final error to overall best and store the overall best
    seqerror = np.empty(niter)
    lowesterror = np.empty(1)
    
    # store overall best factor matrices
    lbest = np.random.rand(data.shape[0], k)
    rbest = np.random.rand(k, data.shape[1])
    
    for l in np.arange(reinit):
        # randomly initialize the factor matrices
        lfactor = np.random.rand(data.shape[0], k)
        rfactor = np.random.rand(k, data.shape[1])
        
        # outer loop for number of iterations 
        for i in np.arange(niter):
            approx = np.matmul(lfactor, rfactor)
            
            # weighted sampling of row and column from data matrix
            row = np.random.choice(data.shape[0], size = 1, p = weightsample(approx, 1))
            col = np.random.choice(data.shape[1], size = 1, p = weightsample(approx, 0))
            #row = np.random.choice(data.shape[0], p = weightsample(approx, 1))
            #col = np.random.choice(data.shape[1], p = weightsample(approx, 0))
            
            # inner loop for number of RK iterations
            for j in np.arange(kacziter):
                # sample indices until at least one nonzero row or col
                
                resample = True
                while(resample):
                    rowsum = 0
                    colsum = 0
                    kaczrow = np.random.choice(lfactor.shape[0], size = s, replace = False)
                    kaczcol = np.random.choice(rfactor.shape[1], size = s, replace = False)

                    for samplerow in kaczrow:
                        rowsum = rowsum + sum(lfactor[samplerow, :])
                    for samplecol in kaczcol:
                        colsum = colsum + sum(rfactor[:, samplerow])
                    if (rowsum > 0 and colsum > 0):
                        resample = False

                # compute BRK step
                #kaczrow = np.random.choice(lfactor.shape[0], size = s, replace = False)
                #kaczcol = np.random.choice(rfactor.shape[1], size = s, replace = False)

                lfactor[row, :] = lfactor[row, :] + np.matmul((data[row, kaczcol] - np.matmul(lfactor[row, :], rfactor[:, kaczcol])), np.linalg.pinv(rfactor[:, kaczcol]))
                rfactor[:, col] = rfactor[:, col] + np.matmul(np.linalg.pinv(lfactor[kaczrow, :]), (data[kaczrow, col, None] - np.matmul(lfactor[kaczrow, :], rfactor[:, col])))

            # calculate error after update
            seqerror[i] = np.linalg.norm(data - np.matmul(lfactor, rfactor)) / np.linalg.norm(data)
        # update after first init
        if (finalerror == -1):
            lowesterror = seqerror
            lbest = lfactor
            rbest = rfactor
        # if not first, only update if final error is lower than overall best
        elif (finalerror > seqerror[niter - 1]):
            finalerror = seqerror[niter - 1]
            lowesterror = seqerror
            lbest = lfactor
            rbest = rfactor
    return(lbest, rbest, lowesterror)

In [8]:
def gs(data, k, niter, gsiter, reinit = 1):
    # set to negative one so we can guarantee an update for the first init
    finalerror = -1
    
    # need to compare final error to overall best and store the overall best
    seqerror = np.empty(niter)
    lowesterror = np.empty(1)
    
    # store overall best factor matrices
    lbest = np.random.rand(data.shape[0], k)
    rbest = np.random.rand(k, data.shape[1])
    
    for l in np.arange(reinit):
        # randomly initialize the factor matrices
        lfactor = np.random.rand(data.shape[0], k)
        rfactor = np.random.rand(k, data.shape[1])
        
        # outer loop for number of iterations 
        for i in np.arange(niter):
            approx = np.matmul(lfactor, rfactor)
            
            # weighted sampling of row and column from data matrix
            #row = np.random.choice(data.shape[0], p = weightsample(approx, 1))
            #col = np.random.choice(data.shape[1], p = weightsample(approx, 0))
            row = np.random.choice(data.shape[0])
            col = np.random.choice(data.shape[1])
            
            # inner loop for number of RK iterations
            for j in np.arange(gsiter):
                # sample indices for entry of data matrix, dont want norms in rk step to be 0
                #gsrow = np.random.choice(rfactor.shape[0], replace = False, p = weightsample(rfactor, 1))
                #gscol = np.random.choice(lfactor.shape[1], replace = False, p = weightsample(lfactor, 0))
                gsrow = np.random.choice(rfactor.shape[0])
                gscol = np.random.choice(lfactor.shape[1])
            
                # compute GS step
                rfactor[:, col] = rfactor[:, col] + np.matmul(lfactor[:, gscol].T, (data[:, col] - np.matmul(lfactor, rfactor[:, col])), np.eye(k)[:, gscol])
                lfactor[row, :] = lfactor[row, :] + np.matmul((data[row, :] - np.matmul(lfactor[row, :], rfactor)), np.matmul(np.array([rfactor[gsrow, :]]).T, np.array([np.eye(k)[gsrow, :]])))
                
            # calculate error after update
            seqerror[i] = np.linalg.norm(data - np.matmul(lfactor, rfactor)) / np.linalg.norm(data)
        # update after first init
        if (finalerror == -1):
            lowesterror = seqerror
            lbest = lfactor
            rbest = rfactor
        # if not first, only update if final error is lower than overall best
        elif (finalerror > seqerror[niter - 1]):
            finalerror = seqerror[niter - 1]
            lowesterror = seqerror
            lbest = lfactor
            rbest = rfactor
    return(lbest, rbest, lowesterror)

In [9]:
def bgs(data, k, s, niter, gsiter, reinit = 1):
    # set to negative one so we can guarantee an update for the first init
    finalerror = -1
    
    # need to compare final error to overall best and store the overall best
    seqerror = np.empty(niter)
    lowesterror = np.empty(1)
    
    # store overall best factor matrices
    lbest = np.random.rand(data.shape[0], k)
    rbest = np.random.rand(k, data.shape[1])
    
    for l in np.arange(reinit):
        # randomly initialize the factor matrices
        lfactor = np.random.rand(data.shape[0], k)
        rfactor = np.random.rand(k, data.shape[1])
        
        # outer loop for number of iterations 
        for i in np.arange(niter):
            approx = np.matmul(lfactor, rfactor)
            
            # weighted sampling of row and column from data matrix
            #row = np.random.choice(data.shape[0], p = weightsample(approx, 1))
            #col = np.random.choice(data.shape[1], p = weightsample(approx, 0))
            row = np.random.choice(data.shape[0], size = 1, p = weightsample(approx, 1))
            col = np.random.choice(data.shape[1], size = 1, p = weightsample(approx, 0))
            
            
            # inner loop for number of RK iterations
            for j in np.arange(gsiter):
                # sample indices for entry of data matrix, dont want norms in rk step to be 0
                #gsrow = np.random.choice(rfactor.shape[0], size = s, replace = False, p = weightsample(rfactor, 1))
                #gscol = np.random.choice(lfactor.shape[1], size = s, replace = False, p = weightsample(lfactor, 0))
                
                if s == 1:
                    gsrow = np.random.choice(rfactor.shape[0], size = s, p = weightsample(rfactor, 1))
                    gscol = np.random.choice(lfactor.shape[1], size = s, p = weightsample(lfactor, 0))

                else:
                    resample = True
                    while(resample):
                        rowsum = 0
                        colsum = 0
                        gscol = np.random.choice(lfactor.shape[1], size = s, replace = False)
                        gsrow = np.random.choice(rfactor.shape[0], size = s, replace = False)

                        for samplerow in gsrow:
                            rowsum = rowsum + sum(lfactor[samplerow, :])
                        for samplecol in gscol:
                            colsum = colsum + sum(rfactor[:, samplerow])
                        if (rowsum > 0 and colsum > 0):
                            resample = False

                # compute BGS step
                lfactor[row, :] = lfactor[row, :] + np.matmul((data[row, :] - np.matmul(lfactor[row, :], rfactor)), np.matmul(np.linalg.pinv(rfactor[gsrow, :]), np.eye(k)[:, gscol].T))
                rfactor[:, col] = rfactor[:, col] + np.matmul(np.matmul(np.eye(k)[:, gscol], np.linalg.pinv(lfactor[:, gscol])), (data[:, col] - np.matmul(lfactor, rfactor[:, col])))
        
            # calculate error after update
            seqerror[i] = np.linalg.norm(data - np.matmul(lfactor, rfactor)) / np.linalg.norm(data)
        # update after first init
        if (finalerror == -1):
            lowesterror = seqerror
            lbest = lfactor
            rbest = rfactor
        # if not first, only update if final error is lower than overall best
        elif (finalerror > seqerror[niter - 1]):
            finalerror = seqerror[niter - 1]
            lowesterror = seqerror
            lbest = lfactor
            rbest = rfactor
    return(lbest, rbest, lowesterror)

In [10]:
def alstest(data, k, niter, reinit = 1):
    A, S, e = als(data, k = k, niter = niter, reinit = reinit)
    approx = np.matmul(A, S)
    return((np.linalg.norm(data - approx) / np.linalg.norm(data)))

In [11]:
def rktest(data, k, niter, kacziter, reinit = 1):
    A, S, error = rk(data, k = k, niter = niter, kacziter = kacziter, reinit = reinit)
    approx = np.matmul(A, S)
    return((np.linalg.norm(data - approx) / np.linalg.norm(data)))

In [12]:
def brktest(data, k, s, niter, kacziter, reinit = 1):
    A, S, error = brk(data, k = k, s = s,  niter = niter, kacziter = kacziter, reinit = reinit)
    approx = np.matmul(A, S)
    return((np.linalg.norm(data - approx) / np.linalg.norm(data)))

In [13]:
def listener(q, textfile):
    '''listens for messages on the q, writes to file. '''
    with open(textfile, 'w') as f:
        while 1:
            m = q.get()
            if m == 'kill':
                f.write('killed')
                break
            f.write(str(m) + ', ')
            f.flush()

In [14]:
def read(filename): 
    with open(filename, 'r') as f:
        l = f.read().split(',')
    return(l)

In [15]:
def alswrite(data, k, niter, q):
    A, S, e = als(data, k = k, niter = niter)
    approx = np.matmul(A, S)
    q.put((np.linalg.norm(data - approx) / np.linalg.norm(data)))

In [16]:
def rkwrite(data, k, niter, kacziter, q):
    A, S, error = rk(data, k = k, niter = niter, kacziter = kacziter)
    approx = np.matmul(A, S)
    q.put((np.linalg.norm(data - approx) / np.linalg.norm(data)))

In [17]:
def brkwrite(data, k, s, niter, kacziter, q):
    A, S, error = brk(data, k = k, s = s, niter = niter, kacziter = kacziter)
    approx = np.matmul(A, S)
    q.put((np.linalg.norm(data - approx) / np.linalg.norm(data)))

In [18]:
def alsmp(data, k, niter, filename, loop, cores = mp.cpu_count()):
    manager = mp.Manager()
    q = manager.Queue()    
    pool = mp.Pool(cores)

    #put listener to work first
    watcher = pool.apply_async(listener, (q, filename))

    #fire off workers
    jobs = []
    for i in range(loop):
        job = pool.apply_async(alswrite, (data, 4, 100, q))
        jobs.append(job)

    # collect results from the workers through the pool result queue
    for job in jobs: 
        job.get()

    #now we are done, kill the listener
    q.put('kill')
    pool.close()
    pool.join()

In [19]:
def rkmp(data, k, niter, kacziter, filename, loop, cores = mp.cpu_count()): 
    manager = mp.Manager()
    q = manager.Queue()    
    pool = mp.Pool(cores)

    #put listener to work first
    watcher = pool.apply_async(listener, (q, filename))

    #fire off workers
    jobs = []
    for i in range(loop):
        job = pool.apply_async(rkwrite, (data, k, niter, kacziter, q))
        jobs.append(job)

    # collect results from the workers through the pool result queue
    for job in jobs: 
        job.get()

    #now we are done, kill the listener
    q.put('kill')
    pool.close()
    pool.join()

In [20]:
def brkmp(data, k, s, niter, kacziter, filename, loop, cores = mp.cpu_count()): 
    manager = mp.Manager()
    q = manager.Queue()    
    pool = mp.Pool(cores)

    #put listener to work first
    watcher = pool.apply_async(listener, (q, filename))

    #fire off workers
    jobs = []
    for i in range(loop):
        job = pool.apply_async(brkwrite, (data, k, s, niter, kacziter, q))
        jobs.append(job)

    # collect results from the workers through the pool result queue
    for job in jobs: 
        job.get()

    #now we are done, kill the listener
    q.put('kill')
    pool.close()
    pool.join()

In [21]:
def extracterr(tag, errfiles): 
    r = re.compile(".*(" + tag + ").*")
    files = list(filter(r.match, errfiles))
    title = list()
    meanerr = list()
    stderr = list()
    for f in reversed(files):
        title.append("".join(re.findall('[0-9]+k*', f)[0]))
        meanerr.append(np.mean(np.asarray(read(f)[:-1]).astype(float)))
        stderr.append(np.std(np.asarray(read(f)[:-1]).astype(float)))
    return(title, meanerr, stderr)

In [22]:
def alsupdate(data, lf, rf, s, siter):
    row = np.random.randint(data.shape[0], size = 1)
    col = np.random.randint(data.shape[1], size = 1)
            
    # perform linear reg update 
    for i in np.arange(siter):
        rf[:, col] = np.matmul(np.linalg.pinv(lf), data[:, col])
        lf[row, :] = np.matmul(data[row, :], np.matmul(rf.T, np.linalg.inv(np.matmul(rf, rf.T))))
    return(lf, rf)

In [23]:
def brkupdate(data, lf, rf, s, siter):
    approx = np.matmul(lf, rf)
            
    # weighted sampling of row and column from data matrix
    # specifying size returns an array rather than a scalar
    row = np.random.choice(data.shape[0], size = 1, p = weightsample(approx, 1))
    col = np.random.choice(data.shape[1], size = 1, p = weightsample(approx, 0))

    # inner loop for number of BRK iterations
    for j in np.arange(siter):
        if s == 1:
            # sample index for entry of data matrix
            kaczrow = np.random.choice(lf.shape[0], size = s, p = weightsample(lf, 1))
            kaczcol = np.random.choice(rf.shape[1], size = s, p = weightsample(rf, 0))
        else:
            resample = True
            while(resample):
                rowsum = 0
                colsum = 0
                kaczrow = np.random.choice(lf.shape[0], size = s, replace = False)
                kaczcol = np.random.choice(rf.shape[1], size = s, replace = False)

                for samplerow in kaczrow:
                    rowsum = rowsum + sum(lf[samplerow, :])
                for samplecol in kaczcol:
                    colsum = colsum + sum(rf[:, samplerow])
                if (rowsum > 0 and colsum > 0):
                    resample = False

            # compute BRK step
    
        lf[row, :] = lf[row, :] + np.matmul((data[None, row, kaczcol] - np.matmul(lf[row, :], rf[:, kaczcol])), np.linalg.pinv(rf[:, kaczcol]))
        rf[:, col] = rf[:, col] + np.matmul(np.linalg.pinv(lf[kaczrow, :]), (data[kaczrow, col, None] - np.matmul(lf[kaczrow, :], rf[:, col])))

    return(lf, rf)

In [62]:
def bgsupdate(data, lf, rf, s, siter):
    approx = np.matmul(lf, rf)
    k = lf.shape[1]
            
    # weighted sampling of row and column from data matrix
    # specifying size returns an array rather than a scalar
    row = np.random.choice(data.shape[0], size = 1, p = weightsample(approx, 1))
    col = np.random.choice(data.shape[1], size = 1, p = weightsample(approx, 0))

    # inner loop for number of GS iterations
    for j in np.arange(siter):
        if s == 1:
            gsrow = np.random.choice(rf.shape[0], size = s, p = weightsample(rf, 1))
            gscol = np.random.choice(lf.shape[1], size = s, p = weightsample(lf, 0))
        else:
            resample = True
            while(resample):
                rowsum = 0
                colsum = 0
                gscol = np.random.choice(lf.shape[1], size = s, replace = False)
                gsrow = np.random.choice(rf.shape[0], size = s, replace = False)

                for samplerow in gsrow:
                    rowsum = rowsum + sum(lf[samplerow, :])
                for samplecol in gscol:
                    colsum = colsum + sum(rf[:, samplerow])
                if (rowsum > 0 and colsum > 0):
                    resample = False

        # compute BGS step
        print(np.matmul((data[row, :] - np.matmul(lf[row, :], rf)), np.matmul(np.linalg.pinv(rf[gsrow, :]), np.eye(k)[:, gscol].T)))
        print(np.matmul(np.matmul(np.eye(k)[:, gscol], np.linalg.pinv(lf[:, gscol])), (data[:, col] - np.matmul(lf, rf[:, col]))))
        lf[row, :] = lf[row, :] + np.matmul((data[row, :] - np.matmul(lf[row, :], rf)), np.matmul(np.linalg.pinv(rf[gsrow, :]), np.eye(k)[:, gscol].T))
        rf[:, col] = rf[:, col] + np.matmul(np.matmul(np.eye(k)[:, gscol], np.linalg.pinv(lf[:, gscol])), (data[:, col] - np.matmul(lf, rf[:, col])))
        #print(np.linalg.norm(np.eye(data.shape[0]) - np.matmul(np.linalg.pinv(np.matmul(lf, rf)), np.matmul(lf, rf))))

    return(lf, rf)

In [57]:
def mf(data, k, s = 1, niter = 100, siter = 1, solver = 'als', errseq = False, reinit = 1):
    # set to negative one so we can guarantee an update for the first init
    finalerror = -1
    rows = np.empty(niter)
    
    # need to compare final error to overall best and store the overall best
    seqerror = np.empty(niter)
    lowesterror = np.empty(1)
    
    # store overall best factor matrices
    lbest = np.random.rand(data.shape[0], k)
    rbest = np.random.rand(k, data.shape[1])
    
    # for bgs, siter > 1 throws an error 
    if solver == "als":
        f = alsupdate
    if solver == "brk":
        f = brkupdate
    if solver == "bgs":
        f = bgsupdate
    
    for l in np.arange(reinit):
        # randomly initialize the factor matrices
        lfactor = np.random.rand(data.shape[0], k)
        rfactor = np.random.rand(k, data.shape[1])
        
        # outer loop for number of iterations 
        for i in np.arange(niter):          
            lfactor, rfactor = f(data, lfactor, rfactor, s, siter)
            
            # calculate error after update
            seqerror[i] = np.linalg.norm(data - np.matmul(lfactor, rfactor)) / np.linalg.norm(data)
        # update after first init
        if (finalerror == -1):
            lowesterror = seqerror
            lbest = lfactor
            rbest = rfactor
        # if not first, only update if final error is lower than overall best
        elif (finalerror > seqerror[-1]):
            finalerror = seqerror[-1]
            lowesterror = seqerror
            lbest = lfactor
            rbest = rfactor
    if (errseq):
        return(lbest, rbest, lowesterror)
    else:
        return(lbest, rbest, lowesterror[-1])

In [26]:
def mfwrite(data, k, s, niter, siter, solver, q, errseq = False, reinit = 1):
    A, S, error = mf(data, k, s, niter, siter, solver, errseq, reinit)
    q.put(error)

In [27]:
def mpmf(data, k, s, niter, siter, solver, filename, loop, cores = mp.cpu_count()):
    manager = mp.Manager()
    q = manager.Queue()    
    pool = mp.Pool(cores)

    #put listener to work first
    watcher = pool.apply_async(listener, (q, filename))

    #fire off workers
    jobs = []
    for i in range(loop):
        job = pool.apply_async(mfwrite, (data, k, s, niter, siter, solver, q))
        jobs.append(job)

    # collect results from the workers through the pool result queue
    for job in jobs: 
        job.get()

    #now we are done, kill the listener
    q.put('kill')
    pool.close()
    pool.join()

In [60]:
sfactor200 = np.random.choice(4, size=(200,4), p=np.array([0.7, 0.1, 0.1, 0.1]))
sweight200 = np.random.randint(0, 2, size=(4, 200))
data = np.matmul(sfactor200, sweight200)

In [63]:
A, S, error = mf(data, k = 4, s = 1, niter = 10, siter = 1000, solver = 'bgs')

[[0.         0.         0.         0.39183316]]
[[0.        ]
 [0.        ]
 [0.        ]
 [0.99936645]]
[[0.         0.         0.         0.04664296]]
[[ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [-1.38777878e-16]]
[[0.00624724 0.         0.         0.        ]]
[[0.26239276]
 [0.        ]
 [0.        ]
 [0.        ]]
[[0.13340814 0.         0.         0.        ]]
[[-8.32667268e-17]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]]
[[-0.10039727  0.          0.          0.        ]]
[[0.]
 [0.]
 [0.]
 [0.]]
[[-1.66851477e-05  0.00000000e+00  0.00000000e+00  0.00000000e+00]]
[[4.16333634e-17]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]]
[[ 0.          0.          0.         -0.20524735]]
[[ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [-0.19928204]]
[[0.         0.         0.         0.15429454]]
[[ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [-1.66533454e-16]]
[[0.         0.         0.         0.15518642]]
[[0.00000000e+00]
 [0.00000000e+00]


[[0.         0.         0.12360376 0.        ]]
[[ 0.        ]
 [ 0.        ]
 [-0.01531242]
 [ 0.        ]]
[[ 0.          0.          0.         -0.22589555]]
[[ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [-0.00654394]]
[[-0.00010303  0.          0.          0.        ]]
[[-0.01026852]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]]
[[0.         0.         0.21155884 0.        ]]
[[0.        ]
 [0.        ]
 [0.02524851]
 [0.        ]]
[[ 0.         -0.15903758  0.          0.        ]]
[[0.        ]
 [0.00770491]
 [0.        ]
 [0.        ]]
[[-0.17226268  0.          0.          0.        ]]
[[-0.02741121]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]]
[[0.         0.07491661 0.         0.        ]]
[[0.        ]
 [0.02824463]
 [0.        ]
 [0.        ]]
[[-0.11242448  0.          0.          0.        ]]
[[-0.02177567]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]]
[[0.10212089 0.         0.         0.        ]]
[[1.38777878e-17]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]]

[[0.        0.        0.        0.1850449]]
[[0.       ]
 [0.       ]
 [0.       ]
 [0.0014176]]
[[0.40652871 0.         0.         0.        ]]
[[0.00466506]
 [0.        ]
 [0.        ]
 [0.        ]]
[[ 0.         -0.49731997  0.          0.        ]]
[[ 0.        ]
 [-0.00582168]
 [ 0.        ]
 [ 0.        ]]
[[-0.00072057  0.          0.          0.        ]]
[[-0.00771989]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]]
[[-0.10806944  0.          0.          0.        ]]
[[6.24500451e-17]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]]
[[-0.03029912  0.          0.          0.        ]]
[[3.46944695e-18]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]]
[[0.         0.         0.57053274 0.        ]]
[[ 0.        ]
 [ 0.        ]
 [-0.01543738]
 [ 0.        ]]
[[ 0.          0.         -0.43512394  0.        ]]
[[ 0.00000000e+00]
 [ 0.00000000e+00]
 [-5.72458747e-17]
 [ 0.00000000e+00]]
[[ 0.          0.          0.         -0.10989661]]
[[0.       ]
 [0.       ]
 [0.  

 [-5.55111512e-17]]
[[ 0.         -0.54460456  0.          0.        ]]
[[0.        ]
 [0.00526365]
 [0.        ]
 [0.        ]]
[[0.38798189 0.         0.         0.        ]]
[[-0.03327986]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]]
[[0.         0.09593803 0.         0.        ]]
[[0.        ]
 [0.01162539]
 [0.        ]
 [0.        ]]
[[0.04148941 0.         0.         0.        ]]
[[-0.00681631]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]]
[[ 0.          0.          0.         -0.04188726]]
[[0.        ]
 [0.        ]
 [0.        ]
 [0.00444719]]
[[0.         0.03044717 0.         0.        ]]
[[0.        ]
 [0.00265494]
 [0.        ]
 [0.        ]]
[[0.         0.         0.         0.00851453]]
[[ 0.       ]
 [ 0.       ]
 [ 0.       ]
 [-0.0011585]]
[[ 0.          0.          0.         -0.43635575]]
[[ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [-2.77555756e-17]]
[[0.         0.35399684 0.         0.        ]]
[[0.        ]
 [0.02155802]
 [0.        ]
 [0.      

[[0.00227239]
 [0.        ]
 [0.        ]
 [0.        ]]
[[0.00000000e+00 3.89511565e-06 0.00000000e+00 0.00000000e+00]]
[[0.        ]
 [0.00051727]
 [0.        ]
 [0.        ]]
[[-0.13382112  0.          0.          0.        ]]
[[-0.0004239]
 [ 0.       ]
 [ 0.       ]
 [ 0.       ]]
[[-0.19547037  0.          0.          0.        ]]
[[-9.71445147e-17]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]]
[[0.         0.         0.23966324 0.        ]]
[[0.        ]
 [0.        ]
 [0.00136536]
 [0.        ]]
[[ 0.          0.          0.         -0.23890448]]
[[0.        ]
 [0.        ]
 [0.        ]
 [0.00117347]]
[[ 0.        -0.0319934  0.         0.       ]]
[[0.        ]
 [0.01123838]
 [0.        ]
 [0.        ]]
[[0.2159155 0.        0.        0.       ]]
[[-0.01462917]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]]
[[0.         0.         0.         0.05876481]]
[[ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [-0.00022048]]
[[0.         0.         0.04556043 0.        ]]
[[ 

...

[[ 0.          0.          0.         -0.94712689]]
[[0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [8.67361738e-18]]
[[0.46399062 0.         0.         0.        ]]
[[0.0124902]
 [0.       ]
 [0.       ]
 [0.       ]]
[[ 0.          0.         -0.65442449  0.        ]]
[[ 0.        ]
 [ 0.        ]
 [-0.00053477]
 [ 0.        ]]
[[ 0.          0.          0.         -0.18263773]]
[[ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [-0.00698745]]
[[-0.03674368  0.          0.          0.        ]]
[[0.00980049]
 [0.        ]
 [0.        ]
 [0.        ]]
[[0.         0.         1.19719186 0.        ]]
[[ 0.       ]
 [ 0.       ]
 [-0.0020219]
 [ 0.       ]]
[[ 0.         -0.86292083  0.          0.        ]]
[[ 0.        ]
 [-0.00490281]
 [ 0.        ]
 [ 0.        ]]
[[ 0.         0.        -0.2634004  0.       ]]
[[ 0.        ]
 [ 0.        ]
 [-0.00242698]
 [ 0.        ]]
[[0.         0.56420624 0.         0.        ]]
[[ 0.        ]
 [-0.00410922]
 [ 0.        ]
 [ 0.        ]]
[[ 

 [0.        ]]
[[0.         2.31282744 0.         0.        ]]
[[ 0.       ]
 [-0.0100316]
 [ 0.       ]
 [ 0.       ]]
[[ 0.          0.         -1.71251696  0.        ]]
[[0.        ]
 [0.        ]
 [0.00306512]
 [0.        ]]
[[0.00000000e+00 0.00000000e+00 8.41461474e-06 0.00000000e+00]]
[[0.00000000e+00]
 [0.00000000e+00]
 [6.07153217e-18]
 [0.00000000e+00]]
[[1.38617687 0.         0.         0.        ]]
[[0.00949917]
 [0.        ]
 [0.        ]
 [0.        ]]
[[0.         0.         0.29231583 0.        ]]
[[ 0.       ]
 [ 0.       ]
 [-0.0004433]
 [ 0.       ]]
[[ 0.          0.          0.         -0.79928953]]
[[ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [-0.00040275]]
[[0.         0.         0.76614099 0.        ]]
[[0.        ]
 [0.        ]
 [0.00043976]
 [0.        ]]
[[ 0.          0.          0.         -1.41978046]]
[[0.       ]
 [0.       ]
 [0.       ]
 [0.0003564]]
[[0.         0.         0.         0.87214563]]
[[ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+

 [ 0.        ]]
[[ 0.          0.          0.         -2.68407206]]
[[0.      ]
 [0.      ]
 [0.      ]
 [0.006602]]
[[0.         1.79559871 0.         0.        ]]
[[0.        ]
 [0.06318902]
 [0.        ]
 [0.        ]]
[[0.         0.         1.98762682 0.        ]]
[[0.        ]
 [0.        ]
 [0.00565289]
 [0.        ]]
[[ 0.          0.         -2.18340764  0.        ]]
[[ 0.0000000e+00]
 [ 0.0000000e+00]
 [-1.6479873e-17]
 [ 0.0000000e+00]]
[[ 0.          0.         -1.59720764  0.        ]]
[[0.00000000e+00]
 [0.00000000e+00]
 [2.68882139e-17]
 [0.00000000e+00]]
[[0.00000000e+00 0.00000000e+00 6.88623623e-07 0.00000000e+00]]
[[0.00000000e+00]
 [0.00000000e+00]
 [1.47451495e-17]
 [0.00000000e+00]]
[[0.87688286 0.         0.         0.        ]]
[[-0.02284067]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]]
[[0.         0.         0.         0.96109924]]
[[ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [-0.01308265]]
[[0.         0.19382225 0.         0.        ]]
[[0.        ]
 [0.01

[[ 0.          0.          0.         -0.18815424]]
[[0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [1.04083409e-17]]
[[ 0.          0.         -0.27969339  0.        ]]
[[ 0.        ]
 [ 0.        ]
 [-0.00295224]
 [ 0.        ]]
[[ 0.          0.         -0.07817761  0.        ]]
[[0.00000000e+00]
 [0.00000000e+00]
 [6.07153217e-18]
 [0.00000000e+00]]
[[ 0.         -0.02183742  0.          0.        ]]
[[0.        ]
 [0.01388813]
 [0.        ]
 [0.        ]]
[[ 0.          0.         -0.00701107  0.        ]]
[[0.        ]
 [0.        ]
 [0.00325655]
 [0.        ]]
[[0.         0.         0.         1.92078882]]
[[ 0.      ]
 [ 0.      ]
 [ 0.      ]
 [-0.007849]]
[[4.96784484e-05 0.00000000e+00 0.00000000e+00 0.00000000e+00]]
[[-0.00112598]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]]
[[ 0.          0.         -1.41087272  0.        ]]
[[0.        ]
 [0.        ]
 [0.00330826]
 [0.        ]]
[[0.         0.         0.         2.82149131]]
[[ 0.00000000e+00]
 [ 0.00000000e+00

 [ 0.00000000e+00]]
[[-2.91118651  0.          0.          0.        ]]
[[0.04193544]
 [0.        ]
 [0.        ]
 [0.        ]]
[[ 0.          0.         -0.00115939  0.        ]]
[[0.        ]
 [0.        ]
 [0.00711494]
 [0.        ]]
[[0.         4.93705695 0.         0.        ]]
[[ 0.        ]
 [-0.04855011]
 [ 0.        ]
 [ 0.        ]]
[[ 0.          0.         -0.29911886  0.        ]]
[[ 0.        ]
 [ 0.        ]
 [-0.00010056]
 [ 0.        ]]
[[ 0.         -0.08535094  0.          0.        ]]
[[0.        ]
 [0.00037611]
 [0.        ]
 [0.        ]]
[[ 0.          0.          0.         -3.15416271]]
[[ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [-0.02532251]]
[[4.30189625 0.         0.         0.        ]]
[[0.06066044]
 [0.        ]
 [0.        ]
 [0.        ]]
[[1.15085347 0.         0.         0.        ]]
[[1.73472348e-17]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]]
[[0.         0.30878795 0.         0.        ]]
[[0.        ]
 [0.03117931]
 [0.        ]
 [

In [51]:
A, S, error = mf(data, k = 4, s = 1, niter = 10, siter = 1000, solver = 'bgs', errseq = True)
print(error)

[0.82372971 0.83849927 0.84501163 4.4763483  4.44162822 4.45065825
 1.01900878 1.02977303 1.21722607 1.66866681]


In [66]:
np.linalg.norm(np.eye(A.shape[0]) - np.matmul(np.linalg.pinv(np.matmul(A, S)), np.matmul(A, S)))

14.0

In [71]:
lfactor = np.random.rand(data.shape[0], 4)
rfactor = np.random.rand(4, data.shape[1])
np.linalg.norm(np.eye(lfactor.shape[0]) - np.matmul(np.linalg.pinv(np.matmul(lfactor, rfactor)), np.matmul(lfactor, rfactor)))

13.999999999999998

In [73]:
np.matmul(np.linalg.pinv(np.matmul(lfactor, rfactor)), np.matmul(lfactor, rfactor))

array([[ 0.01006647,  0.0069343 ,  0.0031056 , ...,  0.00640015,
         0.00751692,  0.00663376],
       [ 0.0069343 ,  0.00953661, -0.00300923, ...,  0.00889978,
         0.00768869,  0.00902666],
       [ 0.0031056 , -0.00300923,  0.01265437, ..., -0.00421445,
         0.00252048, -0.0030961 ],
       ...,
       [ 0.00640015,  0.00889978, -0.00421445, ...,  0.00925231,
         0.00790242,  0.01024711],
       [ 0.00751692,  0.00768869,  0.00252048, ...,  0.00790242,
         0.01126107,  0.01086032],
       [ 0.00663376,  0.00902666, -0.0030961 , ...,  0.01024711,
         0.01086032,  0.0132491 ]])

In [74]:
np.matmul(np.linalg.pinv(np.matmul(A, S)), np.matmul(A, S))

array([[ 2.49958565e-02,  2.47238183e-04,  2.01533692e-02, ...,
         2.49102207e-03, -1.04348010e-02, -2.36928778e-03],
       [ 2.47238183e-04,  8.58452844e-03,  2.90765035e-05, ...,
         1.15869212e-02,  8.35029591e-03,  1.43999321e-02],
       [ 2.01533692e-02,  2.90765035e-05,  1.91201325e-02, ...,
         5.69826621e-03, -5.45004205e-03, -3.74413236e-03],
       ...,
       [ 2.49102207e-03,  1.15869212e-02,  5.69826621e-03, ...,
         3.50074321e-02,  1.26086912e-02,  1.78745564e-02],
       [-1.04348010e-02,  8.35029591e-03, -5.45004205e-03, ...,
         1.26086912e-02,  1.63962059e-02,  1.33906717e-02],
       [-2.36928778e-03,  1.43999321e-02, -3.74413236e-03, ...,
         1.78745564e-02,  1.33906717e-02,  2.53458923e-02]])

In [79]:
k = 4
s = 1
lf = np.random.rand(data.shape[0], k)
rf = np.random.rand(k, data.shape[1])

approx = np.matmul(lf, rf)

# weighted sampling of row and column from data matrix
# specifying size returns an array rather than a scalar
row = np.random.choice(data.shape[0], size = 1, p = weightsample(approx, 1))
col = np.random.choice(data.shape[1], size = 1, p = weightsample(approx, 0))

if s == 1:
    gsrow = np.random.choice(rf.shape[0], size = s, p = weightsample(rf, 1))
    gscol = np.random.choice(lf.shape[1], size = s, p = weightsample(lf, 0))
else:
    resample = True
    while(resample):
        rowsum = 0
        colsum = 0
        gscol = np.random.choice(lf.shape[1], size = s, replace = False)
        gsrow = np.random.choice(rf.shape[0], size = s, replace = False)

        for samplerow in gsrow:
            rowsum = rowsum + sum(lf[samplerow, :])
        for samplecol in gscol:
            colsum = colsum + sum(rf[:, samplerow])
        if (rowsum > 0 and colsum > 0):
            resample = False

# compute BGS step
lf[row, :] = lf[row, :] + np.matmul((data[row, :] - np.matmul(lf[row, :], rf)), np.matmul(np.linalg.pinv(rf[gsrow, :]), np.eye(k)[:, gscol].T))
rf[:, col] = rf[:, col] + np.matmul(np.matmul(np.eye(k)[:, gscol], np.linalg.pinv(lf[:, gscol])), (data[:, col] - np.matmul(lf, rf[:, col])))


In [82]:
np.matmul((data[row, :] - np.matmul(lf[row, :], rf)), np.matmul(np.linalg.pinv(rf[gsrow, :]), np.eye(k)[:, gscol].T))

array([[ 0.        ,  0.        , -0.24859052,  0.        ]])